# ТЕМА 7. Атаки на модели машинного обучения и защита от них
Подготовил студент группы ББМО-01-23 Чурсинов Герман

### Требуется
1. Изучить фреймворк Adversarial Robustness Toolbox (ART) v1.14
2. Рассмотреть основные аспекты, касающиеся классификации атак
3. Выбрать сценарий атаки и реализовать его на платформе `Colab`.


In [1]:
# Устанавливаем дополнительную библиотеку
!pip install adversarial-robustness-toolbox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.9 MB/s eta 0:00:00


In [2]:
# Импортируем нужные библиотеки
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import FastGradientMethod
from art.utils import load_mnist

In [3]:
# Загружаем готовый датасет
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

In [4]:
# Нормализуем данные
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [9]:
# Создаём и обучаем базовую модель
model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.1)

Epoch 1/10
844/844 [==============================] - 4s 4ms/step - loss: 1.5017 - accuracy: 0.6401 - val_loss: 0.7899 - val_accuracy: 0.8457
Epoch 2/10
844/844 [==============================] - 5s 6ms/step - loss: 0.6508 - accuracy: 0.8449 - val_loss: 0.4561 - val_accuracy: 0.8952
Epoch 3/10
844/844 [==============================] - 3s 4ms/step - loss: 0.4667 - accuracy: 0.8794 - val_loss: 0.3541 - val_accuracy: 0.9128
Epoch 4/10
844/844 [==============================] - 3s 4ms/step - loss: 0.3965 - accuracy: 0.8920 - val_loss: 0.3104 - val_accuracy: 0.9158
Epoch 5/10
844/844 [==============================] - 4s 5ms/step - loss: 0.3610 - accuracy: 0.8993 - val_loss: 0.2850 - val_accuracy: 0.9210
Epoch 6/10
844/844 [==============================] - 5s 5ms/step - loss: 0.3384 - accuracy: 0.9043 - val_loss: 0.2690 - val_accuracy: 0.9248
Epoch 7/10
844/844 [==============================] - 3s 4ms/step - loss: 0.3227 - accuracy: 0.9081 - val_loss: 0.2587 - val_accuracy: 0.9255
Epoch 

In [10]:
# Создаем классификатор ART
classifier = TensorFlowV2Classifier(model=model, nb_classes=10, input_shape=(28, 28, 1), loss_object=tf.keras.losses.SparseCategoricalCrossentropy())

In [11]:
# Проводим атаку
attack = FastGradientMethod(estimator=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

In [12]:
# Оцениваем модель
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / y_test.shape[0]
print(f'Точность на адверсальных тестовых примерах: {accuracy * 100:.2f}%')

Точность на адверсальных тестовых примерах: 2.19%


Вывод: полученная модель показала точность в 2.19%.